## В данном блокноте выполняется предобработка датасета

Сокращение длинны контекста

In [1]:
from datasets import load_dataset, load_from_disk #Загрузка датасета 

# ----------- for ds preprocess -----------
import nltk
nltk.data.path.clear()
nltk.data.path.append('/home/kne.21@free.uni-dubna.ru/Desktop/prog/nltk_data')
nltk.download('punkt', download_dir="/home/kne.21@free.uni-dubna.ru/Desktop/prog/nltk_data")
nltk.download('punkt_tab', download_dir="/home/kne.21@free.uni-dubna.ru/Desktop/prog/nltk_data")

print(nltk.data.path)

['/home/kne.21@free.uni-dubna.ru/Desktop/prog/nltk_data']


[nltk_data] Error loading punkt: HTTP Error 403: Forbidden
[nltk_data] Error loading punkt_tab: HTTP Error 403: Forbidden


In [2]:
# ----- | Параметры | -----

OUT_DATA_CACHE = "./prep_datasets" #Директория, с предобработанными датасетами
CACHE_DIR = "../myDoRA_repeat/cache_dir" #Директория с кешем модели
EVAL_MODEL_DIR_OUT = "./eval_models_out" #Для вывода логов

### Dataset - SQuAD 2.0

In [3]:
#Загружаем датасет
datasetSQUAD2 = load_dataset("rajpurkar/squad_v2", cache_dir="./cdatasets")

val_dataset = datasetSQUAD2['validation'] 
train_dataset = datasetSQUAD2['train']

#for test pipeline
small_train_ds = train_dataset.select(range(32))
small_val_ds = val_dataset.select(range(32))

print(val_dataset.info)
print(train_dataset.info)

Using the latest cached version of the dataset since rajpurkar/squad_v2 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'squad_v2' at cdatasets/rajpurkar___squad_v2/squad_v2/0.0.0/3ffb306f725f7d2ce8394bc1873b24868140c412 (last modified on Sat May 17 12:20:11 2025).


DatasetInfo(description='', citation='', homepage='', license='', features={'id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}, post_processed=None, supervised_keys=None, builder_name='parquet', dataset_name='squad_v2', config_name='squad_v2', version=0.0.0, splits={'train': SplitInfo(name='train', num_bytes=116742930, num_examples=130319, shard_lengths=None, dataset_name='squad_v2'), 'validation': SplitInfo(name='validation', num_bytes=11663635, num_examples=11873, shard_lengths=None, dataset_name='squad_v2')}, download_checksums={'hf://datasets/rajpurkar/squad_v2@3ffb306f725f7d2ce8394bc1873b24868140c412/squad_v2/train-00000-of-00001.parquet': {'num_bytes': 16369982, 'checksum': None}, 'hf://datasets/rajpurkar/squad_v2@3ffb306f725f7d2ce

### Предобработка датасета 

Тестировал прямой проход модели на батче из 1 последовательности (1024 токена), работала, но видно с обратным распространением и шагом оптимизатора, памяти не хватает, следовательно,нужно сократить длинну последовательности до 256-512 токенов.

Возьму примерно 256 токенов (контекст обрезается по предложениям, берется текущее предыдушее и следущее).

Инференс на 1024 токена работает хорошо (причем я указываю что использую внимани "eager" для FT и "sdpa" для инференса, но не уверен, что оно корректно используется, не до конца протестировал)

In [4]:
def preprocess_dataset_split_context(batch, neighbors = 1):
    """ Функция для обработки датасета SQuAD 2.0
    Для извлечения context, question, answer. И обрезки context до длинны 4-5 предложений.
    
    Статический срез, берет предыдушее предложение, текущее и следущее (index - neighbors : index + neighbors + 1)

    Args:
        batch : dict - словарь списков с соответствующими ключами dataset'a
    """
    #Есть строки, в которых нет ответов (в контексте не предоставлены ответы)
    #Необходимо, чтобы все ровно соответстовали тому, что было
    contexts = [] #Список контекстов
    questions = [] #Список воспросов
    answers = [] #Список ответов

    for cntxt, quest, answer in zip(batch['context'], batch['question'], batch['answers']):
        cntx_split_sentenses = nltk.sent_tokenize(cntxt, language='english') #Разбиваем контест по предложениям

        if len(answer['text']) > 0: #Ответ есть
            found = False
            for sent_index in range(len(cntx_split_sentenses)):
                if (answer['text'][0] in cntx_split_sentenses[sent_index]): #Если ответ содержится в предложении
                    start = max(0, sent_index - neighbors)
                    end = min(len(cntx_split_sentenses), sent_index + neighbors + 1)
                    res_text_context = " ".join(cntx_split_sentenses[start : end])

                    contexts.append(res_text_context)
                    questions.append(quest)
                    answers.append(answer['text'][0]) #Добавляем именно текст
                    found = True
                    break #ОБЯЗАТЕЛЬНО НУЖНО ВЫЙТИ из цикла
            if not found:
                contexts.append("") #Если не нашли, будет пустой контекст (не будем использовать для обучения и оценки)
                questions.append(quest)
                answers.append(answer['text'][0]) #Добавляем именно текст
        else:
            #Нужно добавить контекст как строку, а не как список ;D
            contexts.append(" ".join(cntx_split_sentenses[:3])) #Берем 3 первых предложения
            questions.append(quest)
            answers.append("") #Добавляем пустую строку
        
    assert len(contexts) == len(batch['context']), f"context not quals size | {len(contexts)} {len(batch['context'])}"
    assert len(questions) == len(batch['question']), f"question not quals size {len(questions)} {len(batch['question'])}"
    assert len(answers) == len(batch['answers']), f"answers not quals size {len(answers)} {len(batch['answers'])}"

    return { "context": contexts, "question": questions, "answers" : answers }

In [5]:
train_short_ds = train_dataset.map(
    preprocess_dataset_split_context, 
    batched=True, 
    num_proc=1,
    remove_columns=train_dataset.column_names,
    cache_file_name="./cdatasets/train_ds_split_contex256.cache"
)

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

In [6]:
val_short_ds = val_dataset.map(
    preprocess_dataset_split_context,
    batched=True,       
    num_proc=1,  
    remove_columns=val_dataset.column_names,  # Удаляем исходные колонки
    cache_file_name="./cdatasets/val_ds_split_contex256.cache"
)

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

### Удалим строки с пустным контекстом

In [7]:
assert len(train_dataset) == len(train_short_ds), "size train_dataset and train_short_ds not equals"
assert len(val_dataset) == len(val_short_ds), "size val_dataset and val_short_ds not equals"
print(len(train_short_ds))
print(len(val_short_ds))

130319
11873


In [8]:
# Подсчитаем сколько пустых контекстов в Train and Val
cnt_train = 0
for short in train_short_ds:
    if short['context'] == "":
        cnt_train += 1

cnt_val = 0
for short in val_short_ds:
    if short['context'] == "":
        cnt_val += 1

print(cnt_train) #214
print(cnt_val) #14

214
14


In [9]:
train_ds_short_filter = train_short_ds.filter(lambda row: False if row['context'] == "" else True)
val_ds_short_filter = val_short_ds.filter(lambda row: False if row['context'] == "" else True)

Filter:   0%|          | 0/130319 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [10]:
print(len(train_ds_short_filter))
print(len(val_ds_short_filter))

print(len(train_short_ds) - len(train_ds_short_filter))
print(len(val_short_ds) - len(val_ds_short_filter))

assert len(train_short_ds) - cnt_train == len(train_ds_short_filter), "Not equal Train"
assert len(val_short_ds) - cnt_val == len(val_ds_short_filter), "Not equal Train"

130105
11859
214
14


In [11]:
train_ds_short_filter[0]

{'context': "Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time.",
 'question': 'When did Beyonce start becoming popular?',
 'answers': 'in the late 1990s'}

In [12]:
val_ds_short_filter[0]

{'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia.',
 'question': 'In what country is Normandy located?',
 'answers': 'France'}

### Сохраним предобработанные датасеты

In [13]:
train_ds_short_filter.save_to_disk(f"{OUT_DATA_CACHE}/train_ds_short")
val_ds_short_filter.save_to_disk(f"{OUT_DATA_CACHE}/val_ds_short")

Saving the dataset (0/1 shards):   0%|          | 0/130105 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11859 [00:00<?, ? examples/s]